**Troubleshooting Scripts - Locking, Blocking and Concurrency**

Dmitri V. Korotkevitch (MCM, MVP)

email: [dk@aboutsqlserver.com](mailto:dk@aboutsqlserver.com)      blog: [https://aboutsqlserver.com](https://aboutsqlserver.com/) code: [https://github.com/aboutsqlserver/code](https://github.com/aboutsqlserver/code)

SQL Server Advanced Troubleshooting and Performance Tuning (O'Reilly, 2022)      ISBN: 978-1098101923

**Blocking Monitoring Framework: [https://github.com/aboutsqlserver/code/bmframework](https://github.com/aboutsqlserver/code/bmframework)**

**Enable Blocked Process Threshold**

Do not set below 5 seconds

In [ ]:
EXEC sp_configure 'show advanced options', 1;
GO
RECONFIGURE;
GO
EXEC sp_configure 'blocked process threshold', 10; -- in seconds
GO
RECONFIGURE;
GO

**Detailed information about all lock requests**

- This needs to be executed in context of database where blocking occurs to resolve object names
- This may be blocked with Schema Modification locks involved due to OBJECT\_NAME() function in the script. You can comment it as needed
- For blockers, SQL and execution plan represent currently running statement rather than the one that caused the blocking. they will be NULL if session is idling
- It filters out locks from the current session (remove WHERE clause when you run it in demos)

In [ ]:
--USE SQLServerInternals
--GO

SELECT
   tl1.resource_type AS [Resource Type]
   ,DB_NAME(tl1.resource_database_id) AS [DB]
   ,CASE tl1.resource_type
      WHEN 'OBJECT' THEN 
         OBJECT_NAME(tl1.resource_associated_entity_id,tl1.resource_database_id)
      WHEN 'DATABASE' THEN
         'DATABASE'
      ELSE
         CASE
            WHEN tl1.resource_database_id = db_id() 
            THEN
            (
               SELECT OBJECT_NAME(object_id,tl1.resource_database_id)
               FROM sys.partitions WITH (NOLOCK)
               WHERE hobt_id = tl1.resource_associated_entity_id
            )
            ELSE
               '(Run under DB context)'
         END
   END AS [Object]
   ,tl1.resource_description AS [Resource]
   ,tl1.request_session_id AS [Session]
   ,tl1.request_mode AS [Mode]
   ,tl1.request_status AS [Status]
   ,wt.wait_duration_ms AS [Wait (ms)]
   ,es.login_time
   ,es.original_login_name
   ,es.host_name
   ,es.program_name
   ,c.client_net_address   
   ,query.sql
   ,query.query_plan
FROM
   sys.dm_tran_locks tl1 WITH (NOLOCK) 
      LEFT OUTER JOIN sys.dm_os_waiting_tasks wt WITH (NOLOCK) ON
         tl1.lock_owner_address =wt.resource_address AND 
         tl1.request_status = 'WAIT'
      LEFT JOIN sys.dm_exec_connections c WITH (NOLOCK) ON 
         tl1.request_session_id = c.session_id 
      LEFT JOIN sys.dm_exec_sessions es WITH (NOLOCK) ON 
         tl1.request_session_id = es.session_id         
      OUTER APPLY
      (
         SELECT
            SUBSTRING(S.text, (er.statement_start_offset/2)+1,
            ((
               CASE er.statement_end_offset
                  WHEN -1 THEN DATALENGTH(S.text)
                  ELSE er.statement_end_offset
               END - er.statement_start_offset)/2)+1
            ) AS [sql]
            ,TRY_CAST(qp.query_plan AS XML) AS query_plan
         FROM 
            sys.dm_exec_requests er WITH (NOLOCK)
               CROSS APPLY sys.dm_exec_sql_text(er.sql_handle) S
               OUTER APPLY sys.dm_exec_text_query_plan
               (
                  er.plan_handle
                  ,er.statement_start_offset
                  ,er.statement_end_offset
               ) qp
         WHERE
            tl1.request_session_id = er.session_id
      ) query
WHERE
   tl1.request_session_id <> @@SPID
ORDER BY
   tl1.request_session_id
OPTION (RECOMPILE, MAXDOP 1);

**Detailed information about blocked and blocking sessions**

- This needs to be executed in context of database where blocking occurs to resolve object names
- This may be blocked with Schema Modification locks involved due to OBJECT\_NAME() function in the script. You can comment it as needed
- For blockers, SQL and execution plan represent currently running statement rather than the one that caused the blocking. they will be NULL if session is idling
- It filters out locks from the current session (remove WHERE clause when you run it in demos)

In [ ]:
--USE SQLServerInternals
--GO

SELECT
   tl1.resource_type AS [Resource Type]
   ,DB_NAME(tl1.resource_database_id) AS [DB]
   ,CASE tl1.resource_type
      WHEN 'OBJECT' THEN 
         OBJECT_NAME(tl1.resource_associated_entity_id,tl1.resource_database_id)
      WHEN 'DATABASE' THEN
         'DATABASE'
      ELSE
         CASE
            WHEN tl1.resource_database_id = db_id() 
            THEN
            (
               SELECT OBJECT_NAME(object_id,tl1.resource_database_id)
               FROM sys.partitions WITH (NOLOCK)
               WHERE hobt_id = tl1.resource_associated_entity_id
            )
            ELSE
               '(Run under DB context)'
         END
   END AS [Object]
   ,tl1.resource_description AS [Resource]
   ,tl1.request_session_id AS [Session]
   ,tl1.request_mode AS [Mode]
   ,tl1.request_status AS [Status]
   ,wt.wait_duration_ms AS [Wait (ms)]
   ,es.login_time
   ,es.original_login_name
   ,es.host_name
   ,es.program_name
   ,c.client_net_address   
   ,query.sql
   ,query.query_plan						
FROM
   sys.dm_tran_locks tl1 WITH (NOLOCK) 
		JOIN sys.dm_tran_locks tl2 WITH (NOLOCK) ON
			tl1.resource_associated_entity_id =
				tl2.resource_associated_entity_id
		LEFT OUTER JOIN sys.dm_os_waiting_tasks wt WITH (NOLOCK) ON
			tl1.lock_owner_address =wt.resource_address AND 
			tl1.request_status = 'WAIT'
		LEFT JOIN sys.dm_exec_connections c WITH (NOLOCK) ON 
			tl1.request_session_id = c.session_id 
		LEFT JOIN sys.dm_exec_sessions es WITH (NOLOCK) ON 
			tl1.request_session_id = es.session_id       	
		OUTER APPLY
		(
			SELECT
				SUBSTRING(S.text, (er.statement_start_offset/2)+1,
				((
					CASE er.statement_end_offset
						WHEN -1 THEN DATALENGTH(S.text)
						ELSE er.statement_end_offset
					END - er.statement_start_offset)/2)+1
				) AS [sql]
				,TRY_CAST(qp.query_plan AS XML) AS query_plan
			FROM 
				sys.dm_exec_requests er WITH (NOLOCK)
				CROSS APPLY sys.dm_exec_sql_text(er.sql_handle) S
				OUTER APPLY sys.dm_exec_text_query_plan
				(
					er.plan_handle
					,er.statement_start_offset
					,er.statement_end_offset
				) qp
			WHERE
				tl1.request_session_id = er.session_id
		) query
WHERE
	tl1.request_session_id <> @@SPID AND
	tl1.request_status <> tl2.request_status AND
	(
		tl1.resource_description = tl2.resource_description OR
		(
			tl1.resource_description IS NULL AND 
			tl2.resource_description IS NULL
		)
	)
ORDER BY
   tl1.request_session_id
OPTION (RECOMPILE, MAXDOP 1);

**Blocking Chains**

In [ ]:
IF OBJECT_ID(N'tempdb..#BlockedSessions') IS NOT NULL
	DROP TABLE #BlockedSessions;
GO

CREATE TABLE #BlockedSessions
(
	session_id INT NOT NULL PRIMARY KEY,
	blocking_session_id INT NULL,
	wait_type SYSNAME NULL, 
	[sql] NVARCHAR(MAX)
);

INSERT INTO #BlockedSessions(session_id, blocking_session_id, wait_type, [sql])
	SELECT 
		er.session_id, er.blocking_session_id, er.wait_type
		,SUBSTRING(
			qt.text, 
	 		(er.statement_start_offset / 2) + 1,
			((CASE er.statement_end_offset
				WHEN -1 THEN datalength(qt.text)
				ELSE er.statement_end_offset
			END - er.statement_start_offset) / 2) + 1
		) 
	FROM 
		sys.dm_exec_requests er WITH (NOLOCK)
			OUTER APPLY sys.dm_exec_sql_text(er.sql_handle) qt
	WHERE 
		er.wait_type LIKE 'LCK_M%';

CREATE INDEX i1 ON #BlockedSessions(blocking_session_id); 

;WITH Locking(level, session_id, blocking_session_id, [sql])
AS
(
	SELECT 0, b.blocking_session_id, null, qt.text as sql
	FROM 
		#BlockedSessions b LEFT JOIN sys.dm_exec_requests er WITH (NOLOCK) ON
			b.blocking_session_id = er.session_id
		OUTER APPLY sys.dm_exec_sql_text(er.sql_handle) qt  
	WHERE
		b.blocking_session_id not in (SELECT session_id from #BlockedSessions)

	UNION ALL
	
	SELECT level + 1, b.session_id, b.blocking_session_id, b.[sql]
	FROM #BlockedSessions b JOIN Locking l ON 
		b.blocking_session_id = l.session_id
)
SELECT 
	l.level, l.session_id, l.blocking_session_id, l.[sql], c.BlockedCnt
FROM 
	Locking l CROSS APPLY
	(
		SELECT COUNT(*) AS BlockedCnt
		FROM #BlockedSessions b 
		WHERE b.blocking_session_id = l.session_id
	) c
ORDER BY l.level, c.BlockedCnt;

SELECT * FROM #BlockedSessions;

**Getting execution plan and query by SQL Handle.**

Set the handle and statement start/end offsets in variables

In [ ]:
DECLARE
   @H VARBINARY(MAX) = 0x00
      /* Insert sql_handle from the top line of the execution stack */
   ,@S INT = 0
      /* Insert stmtStart from the top line of the execution stack */
   ,@E INT = 0
      /* Insert stmtEnd from the top line of the execution stack */

SELECT	
   SUBSTRING(
      qt.text 
      ,(qs.statement_start_offset / 2) + 1
      ,((CASE qs.statement_end_offset
         WHEN -1 THEN DATALENGTH(qt.text)
         ELSE qs.statement_end_offset
      END - qs.statement_start_offset) / 2) + 1
   ) AS SQL
   ,TRY_CAST(qp.query_plan AS XML) AS query_plan
   ,qs.creation_time
   ,qs.last_execution_time
FROM	
   sys.dm_exec_query_stats qs WITH (NOLOCK)
      OUTER APPLY sys.dm_exec_sql_text(qs.sql_handle) qt
      OUTER APPLY sys.dm_exec_text_query_plan(qs.plan_handle,@S,@E) qp
WHERE	
   qs.sql_handle = @H 
OPTION (RECOMPILE, MAXDOP 1);


**Lock Escalations - xEvents session to track** 

Change database\_id in event definition to match the environment

In [ ]:
IF EXISTS (SELECT * FROM sys.server_event_sessions WHERE name = 'LockEscalationTracking')
	DROP EVENT SESSION LockEscalationTracking ON SERVER
GO

CREATE EVENT SESSION LockEscalationTracking
ON SERVER
ADD EVENT
    sqlserver.lock_escalation
    (
        WHERE database_id = 5  -- DB_ID()
    )
ADD TARGET 
    package0.histogram
    (
        SET 
            SLOTS = 1024 -- Based on # of tables in the database
            ,FILTERING_EVENT_NAME = 'sqlserver.lock_escalation'
            ,SOURCE_TYPE = 0 -- event data column
            ,SOURCE = 'object_id' -- grouping column
    )
WITH    
    (
        EVENT_RETENTION_MODE=ALLOW_SINGLE_EVENT_LOSS
        ,MAX_DISPATCH_LATENCY=10 SECONDS
    );

Start xEvent session

In [ ]:
ALTER EVENT SESSION LockEscalationTracking
ON SERVER
STATE=START;

Parse results from xEvent session

In [ ]:
DECLARE
    @X XML;

SELECT @X = CONVERT(XML,st.target_data)
FROM 
    sys.dm_xe_sessions s WITH (NOLOCK)
        JOIN sys.dm_xe_session_targets st WITH (NOLOCK) ON
            s.address = st.event_session_address
WHERE 
    s.name = 'LockEscalationTracking' AND 
    st.target_name = 'histogram'; 

;WITH EventInfo([count],object_id)
as
(
    SELECT
        t.e.value('@count','int') 
        ,t.e.value('((./value)/text())[1]','int') 
    FROM 
        @X.nodes('/HistogramTarget/Slot') as t(e)
)
SELECT 
    e.object_id
    ,s.name + '.' + t.name AS [table]
    ,e.[count]
FROM 
    EventInfo e JOIN sys.tables t WITH (NOLOCK) ON
        e.object_id = t.object_id
    JOIN sys.schemas s WITH (NOLOCK) ON
        t.schema_id = s.schema_id
ORDER BY 
    e.count desc
OPTION (RECOMPILE, MAXDOP 1);

Stop xEvent Session

In [ ]:
ALTER EVENT SESSION LockEscalationTracking
ON SERVER
STATE=STOP;